In [8]:
import numpy as np
import pickle
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [9]:
def plotTSNE(tsne_results, Y_test):
    plt.figure(figsize=(12,12))
    label = Y_test
    colors = ['blue', 'red']
    plt.scatter(x=tsne_results[:,0], y=tsne_results[:,1], c=label, cmap=matplotlib.colors.ListedColormap(colors), s=3)

In [10]:
X_pca_mean = np.asarray(pd.read_csv("data/diabetes/X_pca_mean.csv", header=None))[:6000]
X_val_pca_mean = np.asarray(pd.read_csv("data/diabetes/X_pca_mean.csv", header=None))[6000:8000]
X_test_pca_mean = np.asarray(pd.read_csv("data/diabetes/X_pca_mean.csv", header=None))[8000:]

X_lda_mean = np.asarray(pd.read_csv("data/diabetes/X_lda_mean.csv", header=None))[:6000]
X_val_lda_mean = np.asarray(pd.read_csv("data/diabetes/X_lda_mean.csv", header=None))[6000:8000]
X_test_lda_mean = np.asarray(pd.read_csv("data/diabetes/X_lda_mean.csv", header=None))[8000:]

X_pca_max = np.asarray(pd.read_csv("data/diabetes/X_pca_max.csv", header=None))[:6000]
X_val_pca_max = np.asarray(pd.read_csv("data/diabetes/X_pca_max.csv", header=None))[6000:8000]
X_test_pca_max = np.asarray(pd.read_csv("data/diabetes/X_pca_max.csv", header=None))[8000:]

X_lda_max = np.asarray(pd.read_csv("data/diabetes/X_lda_max.csv", header=None))[:6000]
X_val_lda_max = np.asarray(pd.read_csv("data/diabetes/X_lda_max.csv", header=None))[6000:8000]
X_test_lda_max = np.asarray(pd.read_csv("data/diabetes/X_lda_max.csv", header=None))[8000:]

X_pca_conc = np.asarray(pd.read_csv("data/diabetes/X_pca_conc.csv", header=None))[:6000]
X_val_pca_conc = np.asarray(pd.read_csv("data/diabetes/X_pca_conc.csv", header=None))[6000:8000]
X_test_pca_conc = np.asarray(pd.read_csv("data/diabetes/X_pca_conc.csv", header=None))[8000:]

X_lda_conc = np.asarray(pd.read_csv("data/diabetes/X_lda_conc.csv", header=None))[:6000]
X_val_lda_conc = np.asarray(pd.read_csv("data/diabetes/X_lda_conc.csv", header=None))[6000:8000]
X_test_lda_conc = np.asarray(pd.read_csv("data/diabetes/X_lda_conc.csv", header=None))[8000:]

Y = np.asarray(pd.read_csv("data/diabetes/Y.csv", header=None)).reshape(-1).astype(int)
Y_val = np.asarray(pd.read_csv("data/diabetes/Y_val.csv", header=None)).reshape(-1).astype(int)
Y_test = np.asarray(pd.read_csv("data/diabetes/Y_test.csv", header=None)).reshape(-1).astype(int)
header = pd.read_csv("data/diabetes/diab_train.csv").keys()

In [11]:
X = X_lda_conc
X_val = X_val_lda_conc
X_test = X_test_lda_conc

In [5]:
X2 = np.vstack([X, X_val])
Y2 = np.hstack([Y, Y_val])

In [6]:
print(header)

Index(['Unnamed: 0', 'race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'payer_code', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide.metformin', 'glipizide.metformin',
       'glimepiride.pioglitazone', 'metformin.rosiglitazone',
       'metformin.pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'diag_1_desc', 'diag_2_desc', 'diag_3_desc'],
      dtype='object')


In [15]:
import lightgbm as lgbm
param_grid = {
    'boosting_type': ['gbdt', 'dart'],
    'num_leaves': [11, 13, 15],
    'reg_alpha': [0.6, 0.7, 0.8, 0.9],
    'min_data_in_leaf': [140, 150, 160, 170, 180],
    'max_bin': [300, 350, 400, 450, 500],
    'feature_fraction': [.2, .4, .5, .8]
    }

lgb_estimator = lgbm.LGBMClassifier(objective='binary', 
                                    class_weight='balanced',
                                    eval_metric='f1',
                                    jobs=-1)
gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=X2, y=Y2)
gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=gkf, verbose=3, n_jobs=6, scoring='f1')

In [16]:
lgb_model = gsearch.fit(X=X2, y=Y2)
print(lgb_model.best_params_, lgb_model.best_score_)

Fitting 5 folds for each of 2400 candidates, totalling 12000 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    2.2s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:    6.2s
[Parallel(n_jobs=6)]: Done 276 tasks      | elapsed:   13.7s
[Parallel(n_jobs=6)]: Done 500 tasks      | elapsed:   25.7s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:   39.8s
[Parallel(n_jobs=6)]: Done 1140 tasks      | elapsed:   57.2s
[Parallel(n_jobs=6)]: Done 1556 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done 2036 tasks      | elapsed:  1.9min
[Parallel(n_jobs=6)]: Done 2580 tasks      | elapsed:  2.5min
[Parallel(n_jobs=6)]: Done 3188 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 3860 tasks      | elapsed:  4.1min
[Parallel(n_jobs=6)]: Done 4596 tasks      | elapsed:  5.3min
[Parallel(n_jobs=6)]: Done 5396 tasks      | elapsed:  6.9min
[Parallel(n_jobs=6)]: Done 6260 tasks      | elapsed:  8.5min
[Parallel(n_jobs=6)]: Done 7188 tasks      | elapsed:  9.9min


{'boosting_type': 'gbdt', 'feature_fraction': 0.8, 'max_bin': 300, 'min_data_in_leaf': 140, 'num_leaves': 13, 'reg_alpha': 0.6} 0.5810963450825295


In [13]:
lgb_estimator = lgbm.LGBMClassifier(boosting_type='gbdt',
                                    min_data_in_leaf=170, \
                                    num_leaves=13, 
                                    max_bin=300, \
                                    objective='binary', \
                                    class_weight='balanced',
                                    feature_fraction=.2,
                                    num_boost_round=1000,
                                    reg_alpha=.9
                                   )
lgb_estimator.fit(np.vstack(X2),
                  np.hstack(Y2),
                  eval_set=(X_test, Y_test), 
                  early_stopping_rounds=5, 
                  eval_metric='f1', verbose=0, 
                  callbacks = [lgbm.reset_parameter(learning_rate = np.linspace(0.12, 0.05, 20).tolist()+[0.05]*980)])

C:\Users\Rafael\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, feature_fraction=0.2,
               importance_type='split', learning_rate=0.1, max_bin=300,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_data_in_leaf=170, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_boost_round=1000, num_leaves=13,
               objective='binary', random_state=None, reg_alpha=0.9,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [14]:
print(f1_score(Y_test, lgb_estimator.predict(X_test).round().astype(int)))
print(lgb_estimator.predict(X_test)[:30])
print((np.round(np.max(lgb_estimator.predict_proba(X_test)[:30], axis=1), 1)*10).astype(int))
print(Y_test[:30])

0.5758477096966093
[1 1 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1]
[7 5 8 7 6 6 7 8 5 6 6 6 6 7 6 5 7 6 6 6 6 5 5 6 7 6 6 6 7 7]
[0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0]


In [45]:
X2.shape

(8000, 60)

In [46]:
np.argsort(lgb_estimator.feature_importances_)[::-1]

array([15, 20,  2,  5, 31, 59,  7, 17, 43, 51, 52, 41, 21, 44,  6, 30, 32,
       42, 50, 35, 14, 37, 48, 40, 49, 39, 36, 56, 46, 55,  3,  4,  0, 54,
       38, 33, 27, 47, 58, 34, 45, 13, 57, 16, 53, 10, 26, 18, 19, 28, 29,
       23,  1, 25, 22, 24, 12,  8, 11,  9], dtype=int64)

In [47]:
lgb_estimator.feature_importances_.shape

(60,)

In [31]:
from sklearn.svm import SVC

In [32]:
param_grid = {
    'C': [30, 35, 40, 45, 50],
    'kernel': ['linear', 'rbf'],
    #'degree': [2, 3, 4, 5],
    'gamma': ['auto'],
    'shrinking': [True],
    'decision_function_shape': ['ovo', 'ovr']
    }

clf = SVC(class_weight='balanced')

gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=X2, y=Y2)
gsearch_clf = GridSearchCV(estimator=clf, param_grid=param_grid, cv=gkf, verbose=3, n_jobs=-1, scoring='f1')

In [33]:
clf_model = gsearch_clf.fit(X=X2, y=Y2)
print(clf_model.best_params_, clf_model.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   31.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.7min finished


{'C': 50, 'decision_function_shape': 'ovo', 'gamma': 'auto', 'kernel': 'linear', 'shrinking': True} 0.5200318896168874


In [37]:
clf = SVC(class_weight='balanced', C=50, kernel='linear', degree=2, gamma='auto', shrinking=True, decision_function_shape='ovo')
clf.fit(X2, Y2)

SVC(C=50, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovo', degree=2, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [39]:
f1_score(Y_test, clf.predict(X_test).round().astype(int))

0.4872298624754421

In [42]:
from sklearn.ensemble import AdaBoostClassifier

In [35]:
param_grid = {
    'C': [0.1, 0.5, 1, 2],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4, 5],
    'gamma': ['auto', 'scale'],
    'shrinking': [True, False],
    'decision_function_shape': ['ovo', 'ovr']
    }

ada = AdaBoostClassifier(n_estimators=1000, random_state=0)

gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=X2, y=Y2)
gsearch_ada = GridSearchCV(estimator=ada, param_grid=param_grid, cv=gkf, verbose=3, n_jobs=-1, scoring='f1')

In [43]:
ada_model = gsearch_ada.fit(X=X2, y=Y2)
print(ada_model.best_params_, ada_model.best_score_)

Fitting 0 folds for each of 512 candidates, totalling 0 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   0 out of   0 | elapsed:    0.0s finished


ValueError: No fits were performed. Was the CV iterator empty? Were there no candidates?

In [62]:
ada = AdaBoostClassifier(n_estimators=1000, random_state=0)
ada.fit(X2, Y2)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=1000, random_state=0)

In [63]:
f1_score(Y_test, ada.predict(X_test))

0.4612037708484409

In [46]:
from sklearn.neighbors import KNeighborsClassifier

In [48]:
param_grid = {
    'n_neighbors': [3, 4, 5, 6, 10, 15],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    }

knn = KNeighborsClassifier(4, weights='distance')

gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=X2, y=Y2)
gsearch_knn = GridSearchCV(estimator=knn, param_grid=param_grid, cv=gkf, verbose=3, n_jobs=-1, scoring='f1')

In [49]:
knn_model = gsearch_knn.fit(X=X2, y=Y2)
print(knn_model.best_params_, knn_model.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   17.7s


{'algorithm': 'auto', 'n_neighbors': 4, 'weights': 'distance'} 0.42078240224122226


[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   30.5s finished


In [50]:
knn = KNeighborsClassifier(4, weights='distance')
knn.fit(X2, Y2)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='distance')

In [51]:
f1_score(Y_test, knn.predict(X_test))

0.4282933160285529

In [52]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [53]:
kernel = 1.0 * RBF(1.0)

In [55]:
gpc = GaussianProcessClassifier(kernel=kernel, random_state=0, n_jobs=-1).fit(X2, Y2)

KeyboardInterrupt: 

In [ ]:
f1_score(Y_test, gpc.predict(X_test))

In [ ]:
from sklearn import tree

In [ ]:
dt = tree.DecisionTreeClassifier()
dt = dt.fit(X2, Y2)

In [ ]:
f1_score(Y_test, dt.predict(X_test))